In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

from scipy.fftpack import rfft, fftfreq, fft
import scipy.sparse as sparse
import scipy.linalg as linalg
from scipy.sparse.linalg import LinearOperator
import numpy as np
import collections
import copy
from amfe.utils.utils import OrderedSet
from amfe.cyclic.cyclic import SelectionOperator
import amfe
from amfe.frequency_module import frequency
from scipy import optimize
#unpack case dict

msh_file = r'meshes\2D_beam_20_elem.msh'
m = amfe.Mesh()
m.import_msh(msh_file)

ax= amfe.plot_mesh(m)
ax.axis('equal')
ax.legend(loc=2)

In [ ]:
m.get_phys_group_types()

In [ ]:
# creating a mechanical component
my_comp = amfe.MechanicalSystem()
my_comp.set_mesh_obj(m)

rho = 7.86E-9 # kg/mm3
E = 210.0E3 # MPa = N/mm2
my_material = amfe.KirchhoffMaterial(E=E, nu=0.3, rho=rho, thickness=1, plane_stress=True)
my_comp.set_domain(6,my_material)


print('Number of nodes is equal to %i' %my_comp.mesh_class.no_of_nodes)

In [ ]:
dirsub = m.get_submesh('phys_group', 1)
neusub = m.get_submesh('phys_group', 12)
contactsub = m.get_submesh('phys_group', 11)
domain = m.get_submesh('phys_group', 6)
id_matrix = my_comp.assembly_class.id_matrix

# get dofs
all_dofs = OrderedSet(my_comp.get_dofs(domain,direction ='xy'))
dir_dofs = OrderedSet(my_comp.get_dofs(dirsub, direction ='xy'))
neu_dofs = OrderedSet(my_comp.get_dofs(neusub, direction ='y'))
contact_dofs = OrderedSet(my_comp.get_dofs(contactsub, direction ='y'))


boundary_dofs = dir_dofs | neu_dofs | contact_dofs
interior_dofs = list(OrderedSet(all_dofs) - boundary_dofs)
dir_dofs = list(dir_dofs)
neu_dofs = list(neu_dofs)
contact_dofs = list(contact_dofs)

dof_dict = collections.OrderedDict()
dof_dict['d'] = dir_dofs 
dof_dict['n'] = neu_dofs
dof_dict['c'] = contact_dofs
dof_dict['i'] = interior_dofs

s = SelectionOperator(dof_dict,id_matrix)


In [ ]:
K, f = my_comp.assembly_class.assemble_k_and_f()
M = my_comp.assembly_class.assemble_m()

val = -10
f = val*s.build_B('n').toarray().flatten()

In [ ]:
K_red = s.assemble_matrix(K,['n','c','i'])
M_red = s.assemble_matrix(M,['n','c','i'])
f_red = s.assemble_vector(f,['n','c','i'])

In [ ]:
u_red = sparse.linalg.spsolve(K_red,f_red)
u_dir = np.array(len(s.selection_dict['d'])*[0])

u = np.append(u_dir,u_red)
u_sol = s.P.T.dot(u) # back to original order
my_comp.u_output.append(0*u_sol)
my_comp.u_output.append(u_sol)

In [ ]:
from ipywidgets import interact
import mpl_toolkits.mplot3d as a3


fig, ax1 = plt.subplots(1)
@interact(factor=(-1,1,0.1))
def plot(factor=1):
    ax1.clear()
    amfe.plot_2D_system_solution(my_comp,factor=100*factor,ax=ax1)
    ax1.axis('equal')
    

In [ ]:
# Harmonic Analysis
Bc = s.build_B('c')
f0 = 10
Z_red = frequency.create_Z_matrix(K_red,0*K_red,M_red,f0=f0,complex_data= False, static=False)

u_dym_red = sparse.linalg.spsolve(K_red,f_red)
u_dir = np.array(len(s.selection_dict['d'])*[0])
u = np.append(u_dir,u_red)
u_dym = s.P.T.dot(u) # back to original order


time_list = list(np.arange(0,1/(f0), 0.0005))
u_time_list = list(map(lambda t : np.cos(2.0*np.pi*f0*t)*u_dym, np.arange(0,1/(f0), 0.0005)))
my_comp.u_output = []
my_comp.u_output.extend(u_time_list)


In [ ]:
from matplotlib.animation import FuncAnimation
import matplotlib.animation as animation
fig2, (ax21,ax22) = plt.subplots(1,2,figsize=(10,5))
#fig2, ax21 = plt.subplots(1,1,figsize=(10,5))
ax21.set_ylim([-4,6])
ax21.set_xlim([0,10])
ax21.axis('equal')

def update(u_id=0):
    global ax21
    ax21.clear()
    ax22.clear()
    contact_dof = s.selection_dict['c'][0]
    u_contact_disp = u_time_list[u_id][contact_dof]
    t = time_list[:u_id+1]
    x = np.array(u_time_list).T[contact_dof][:u_id+1]
    ax22.plot(t,x,'r-')
    ax22.plot(t[u_id],u_contact_disp,'yo')
    l, ax21 = amfe.plot_2D_system_solution(my_comp,u_id=u_id,factor=100,ax=ax21,highlight_nodes=[1])
    #ax2.axis('equal')
    ax22.set_xlabel('time (s)')
    ax22.set_ylabel('Y displacement (mm)')
    ax22.set_ylim([-0.1,0.1])
    ax22.set_xlim([0,0.1])
    
    return l
    
ani = FuncAnimation(fig2, update, frames=np.arange(0, len(u_time_list) ,1), blit=True, interval=1)    
#update(len(u_time_list)-1)

In [ ]:
def cubic_string(beta):
    
    return lambda u : beta*(u**3)

def build_nonlinear_force_in_freq(fnl,Q,Bc):
    
    return lambda u_ : Bc.T.dot(Bc.dot(Q.T.dot((fnl(Q.dot(u_))))))
    
def build_nonlinear_force_in_freq_with_Q_red(fnl,Q_red):
    
    return lambda u_ : Q_red.T.dot((fnl(Q_red.dot(u_))))
    
def build_residual(Z,fnl_,fl_):
    ''' 
    Parameters
        Z : np.array
            Dynamic stiffness matrix
        fnl : lambda function
            
        
    '''
    
    return lambda u_ : Z.dot(u_) + fnl_(u_) - fl_

# Harmonic Analysis


Bc_red = s.reduced_selector.build_B('c')
Bn_red = s.reduced_selector.build_B('n')

f0 = 29.85
number_of_harm=20
value = -10000
n_points = 300
beta = 100.0

force_in_time = frequency.linear_harmonic_force(a=value, f0 = f0, n_points=n_points, cos=True)
q = frequency.cos_bases_for_MHBM(f0,number_of_harm=number_of_harm,n_points=n_points)
fl_1dof_ = q.T.dot(force_in_time) # creating 1 dof force in Frequency domain
Z_red = frequency.create_Z_matrix(K_red,0*K_red,M_red,f0=f0,nH=number_of_harm, complex_data= False, static=False)


u_dym_red = sparse.linalg.spsolve(K_red,f_red)


# building multidimentional Harmonic Basis
amplitute_dim = K_red.shape[0] 
I = sparse.eye(amplitute_dim)

Q = sparse.kron(I,q)
I_harm = sparse.eye(number_of_harm)
Bc_aug = sparse.kron(I_harm, Bc_red)
Bn_aug = sparse.kron(I_harm, Bn_red)
Q_red = sparse.kron(Bc_red ,q)
#fl_aug_ = sparse.kron(I_harm, fl_)


fnl = cubic_string(beta)
fnl_ = build_nonlinear_force_in_freq(fnl,Q,Bc_aug)
fnl2_ = build_nonlinear_force_in_freq_with_Q_red(fnl,Q_red)
fl_ = Bn_aug.T.dot(fl_1dof_) # creating multiple dofs force in Frequency domain
R_func = build_residual(Z_red,fnl_,fl_)

#ul_ = sparse.linalg.spsolve(Z_red,fl_)
u_ = np.random.rand(Z_red.shape[0])
R_func(u_)
opt_obj = optimize.root(R_func,u_,method='krylov')
x0 = opt_obj.x

In [ ]:
#I_red = sparse.hstack([I]*number_of_harm) # create matrix for sumation of amplitudes
u_dir = np.array(len(s.selection_dict['d'])*[0])
get_amplitute_by_t_id = lambda t_id : sparse.kron(q[t_id,:],I).dot(x0)

u_ndym = []
for t_id in range(n_points):
    unl_ = get_amplitute_by_t_id(t_id)
    u = np.append(u_dir,unl_)    
    u_ndym.append(s.P.T.dot(u)) # back to original order

#time_list = list(np.arange(0,1/(f0), 0.0005))
#u_time_list = list(map(lambda t : np.cos(2.0*np.pi*f0*t)*u_ndym, np.arange(0,1/(f0), 0.0005)))


In [ ]:
my_comp.u_output = []
my_comp.u_output.extend(u_ndym)

from matplotlib.animation import FuncAnimation
import matplotlib.patches as mpatches
import matplotlib.animation as animation
fig3, (ax31,ax32) = plt.subplots(1,2,figsize=(10,5))
#fig2, ax21 = plt.subplots(1,1,figsize=(10,5))
ax31.set_ylim([-4,6])
ax31.set_xlim([0,10])
ax31.axis('equal')


contact_dof = s.selection_dict['c'][0]
contact_dof_in_time = 5*np.array(my_comp.u_output).T[contact_dof]/300
t = list(range(0,len(contact_dof_in_time)))
u_max = max(contact_dof_in_time)
def update2(u_id=0):
    global ax31
    global value
    ax31.clear()
    ax32.clear()
    
    u_contact_disp = contact_dof_in_time[u_id]
    mult = 5000/abs(value) 
    x =  1.0*contact_dof_in_time[:u_id+1]
    time_slice = t[:u_id+1]
    ax32.plot(time_slice,x,'r-')
    #ax32.plot(contact_dof_in_time,'r-')
    ax32.plot(time_slice[-1],x[-1],'yo')
    l, ax31 = amfe.plot_2D_system_solution(my_comp,u_id=u_id,factor=mult, ax=ax31, highlight_nodes=[1])
    red_patch = mpatches.Patch(color='y', label='Nonlinear DoF')
    ax31.legend(handles=[red_patch])
    #ax2.axis('equal')
    ax32.set_xlabel('time increament')
    ax32.set_ylabel('Y displacement [mm]')
    ax32.set_ylim([-u_max*1.5,u_max*1.5])
    ax32.set_xlim([0,len(contact_dof_in_time)])
    
    ax31.set_xlabel('X [mm]')
    ax31.set_ylabel('Y [mm]')
    return l
    
plt.subplots_adjust(wspace=0.3)
ani = FuncAnimation(fig3, update2, frames=np.arange(0, len(contact_dof_in_time) ,5), blit=True, interval=1)    
#update2(len(contact_dof_in_time)-1)

#fig3.savefig('HBM_beam.png')

In [ ]:
N = n_points
T = 1/f0
dt = T/(N-1)
fft_obj = frequency.Fourier(T,dt)
fft_obj.freq_list

plt.figure()
plt.plot(fft_obj.freq_list,np.abs(fft_obj.fourier_transform(contact_dof_in_time)),'b--')
plt.xlabel('Time points')
plt.ylabel('Amplitudes')
plt.title('Discrite Fourier Transform of the Harmonic Response')